In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import Utils.hedging_utils as hu
import Utils.greeks as gr
import Utils.option_pricing_utils as opu

In [2]:
option_exposure = -100000  # Number of options held in portfolio that we want to hedge
S0 = 49
r = 0.05
sigma = 0.2
#sigma2 = 0.2 # Realized vol of the path (greater than vol used in BS price - we underestimate)
q = 0
T = 20/52 #20/52
K = 50
type = 'c'
M=10000
N=10

In [3]:
# True BS Price
bs_price = opu.get_bs_price_analytical(S0, K, T, sigma, r, q, type, display_result=True)
bs_price

The time of execution of is: 0.0032927989959716797
Option price is 2.4005273232717137


2.4005273232717137

In [4]:
discounted_hedging_cost_array = np.array([hu.simulate_delta_hedge(S0, K, T, sigma, r, q, type, option_exposure, M) for x in range(N)])

c:\Users\Matthew\Documents\Python Scripts\Option_Pricing\Utils\option_pricing_utils.py:7: RuntimeWarning: divide by zero encountered in true_divide
  d1 = (np.log(S/K) + (r-q+sigma**2/2)*T)/(sigma*np.sqrt(T))


In [ ]:
np.mean(discounted_hedging_cost_array)

,Stock_Price,t,Delta,Shares_Held,Shares_Purchased,Cost_of_Shares_Purchased,Shares_Cumulative_Cost,Interest_Cost,Cumulative_Cost_Inc_Interest
0,119871.060669,119846.752977,119846.821472,145926.793723,145926.793723,1.397778e+06,1.397778e+06,119846.560669,1.397778e+06
1,119871.054927,119846.752958,119846.821091,145888.699791,119808.466738,1.179852e+05,1.395917e+06,119849.018476,1.395919e+06
2,119871.053501,119846.752939,119846.820990,145878.663827,119836.524705,1.193764e+05,1.395446e+06,119849.014896,1.395451e+06
3,119871.056739,119846.752919,119846.821198,145899.396453,119867.293295,1.208738e+05,1.396474e+06,119849.013991,1.396481e+06
4,119871.062130,119846.752900,119846.821546,145934.226407,119881.390624,1.215615e+05,1.398189e+06,119849.015967,1.398198e+06
...,...,...,...,...,...,...,...,...,...
9995,119869.679369,119846.560746,119846.660669,129846.575562,119846.569602,1.198470e+05,7.232273e+05,119847.721132,7.420163e+05
9996,119869.671108,119846.560727,119846.660669,129846.560683,119846.545790,1.198458e+05,7.232265e+05,119847.721133,7.420167e+05
9997,119869.652656,119846.560708,119846.660669,129846.560669,119846.560656,1.198466e+05,7.232265e+05,119847.721132,7.420178e+05
9998,119869.637120,119846.560689,119846.660669,129846.560669,119846.560669,1.198466e+05,7.232265e+05,119847.721132,7.420190e+05


In [ ]:
discounted_hedging_cost_array

array([[240417.68178756532,
              Stock_Price         t     Delta    Shares_Held  Shares_Purchased  \
        0       49.000000  0.384615  0.521605   52160.466107      52160.466107
        1       49.032115  0.384577  0.523701   52370.117049        209.650942
        2       49.100147  0.384538  0.528144   52814.431481        444.314432
        3       49.192872  0.384500  0.534188   53418.774856        604.343375
        4       49.269113  0.384462  0.539142   53914.173009        495.398153
        ...           ...       ...       ...            ...               ...
        9995    52.444714  0.000154  1.000000  100000.000000         -0.000000
        9996    52.520420  0.000115  1.000000  100000.000000         -0.000000
        9997    52.501888  0.000077  1.000000  100000.000000         -0.000000
        9998    52.471548  0.000038  1.000000  100000.000000         -0.000000
        9999    52.466687  0.000000  1.000000  100000.000000         -0.000000

              Cost_o

In [7]:
#S = np.array([49,48.12,47.37,50.25,51.75,53.12,53,51.87,51.38,53,49.88,48.5,49.88,50.37,52.13,51.88,52.87,54.87,54.62,55.87,57.25])
S=np.array([49,49.75,52,50,48.38,48.25,48.75,49.63,48.25,48.25,51.12,51.50,49.88,49.88,48.75,47.5,48,46.25,48.13,46.63,48.12])
M=len(S)
dt = 1/52
T_array = np.linspace(T, 0, 21)


In [8]:
# Calculate option Greeks
delta_array = gr.get_delta(S, K, T_array, sigma, r, q, type)
delta_array = np.round(delta_array, 3)

c:\Users\Matthew\Documents\Python Scripts\Option_Pricing\Utils\option_pricing_utils.py:7: RuntimeWarning: divide by zero encountered in true_divide
  d1 = (np.log(S/K) + (r-q+sigma**2/2)*T)/(sigma*np.sqrt(T))


In [9]:
# Simulate hedging
summary_df = pd.DataFrame(list(zip(S, T_array, delta_array)), columns=['Stock_Price', 't', 'Delta'])
summary_df['Shares_Held'] = -option_exposure*delta_array
summary_df['Shares_Purchased'] = -summary_df['Shares_Held'].sub(summary_df['Shares_Held'].shift(-1).fillna(0)).shift().fillna(-summary_df['Shares_Held'])
summary_df['Cost_of_Shares_Purchased'] = summary_df['Shares_Purchased'].mul(summary_df['Stock_Price']).round(-2)
summary_df['Shares_Cumulative_Cost'] = summary_df['Cost_of_Shares_Purchased'].cumsum()
summary_df['Interest_Cost'] = (summary_df['Shares_Cumulative_Cost']*(r*dt)).shift().fillna(0).round(-2)
summary_df['Cumulative_Cost_Inc_Interest'] = np.nan
summary_df['Cumulative_Cost_Inc_Interest'].iloc[0] = summary_df['Shares_Cumulative_Cost'].iloc[0]
for i in range(1, M):
    summary_df.loc[i, 'Cumulative_Cost_Inc_Interest'] = summary_df.loc[i-1, 'Cumulative_Cost_Inc_Interest'] + summary_df.loc[i, 'Cost_of_Shares_Purchased']+ summary_df.loc[i, 'Interest_Cost']
summary_df


,Stock_Price,t,Delta,Shares_Held,Shares_Purchased,Cost_of_Shares_Purchased,Shares_Cumulative_Cost,Interest_Cost,Cumulative_Cost_Inc_Interest
0,49.00,0.384615,0.522,52200.0,52200.0,2557800.0,2557800.0,0.0,2557800.0
1,49.75,0.365385,0.568,56800.0,4600.0,228800.0,2786600.0,2500.0,2789100.0
2,52.00,0.346154,0.705,70500.0,13700.0,712400.0,3499000.0,2700.0,3504200.0
3,50.00,0.326923,0.579,57900.0,-12600.0,-630000.0,2869000.0,3400.0,2877600.0
4,48.38,0.307692,0.459,45900.0,-12000.0,-580600.0,2288400.0,2800.0,2299800.0
5,48.25,0.288462,0.443,44300.0,-1600.0,-77200.0,2211200.0,2200.0,2224800.0
6,48.75,0.269231,0.475,47500.0,3200.0,156000.0,2367200.0,2100.0,2382900.0
7,49.63,0.250000,0.540,54000.0,6500.0,322600.0,2689800.0,2300.0,2707800.0
8,48.25,0.230769,0.420,42000.0,-12000.0,-579000.0,2110800.0,2600.0,2131400.0
9,48.25,0.211538,0.410,41000.0,-1000.0,-48200.0,2062600.0,2000.0,2085200.0


In [10]:
hedging_cost = summary_df['Cumulative_Cost_Inc_Interest'].iloc[-1] - summary_df['Shares_Held'].iloc[-1]*K

In [11]:
hedging_cost

255900.0

In [12]:
discounted_hedging_cost = np.exp(-r*T)*hedging_cost
discounted_hedging_cost

251025.86306797588